In [ ]:
import scipy
from pathlib import Path
from typing import Sequence

import numpy as np
from matplotlib import pyplot as plt

from src.plot_scripts.common import get_parameters
from out.plots import get_plots_path
from out.results import get_results_path
from out.cache import get_cache_path, set_cache_path
from out.mat import get_mat_path

In [ ]:
basepath = get_results_path()
# slurm_id = 2041250
# basepath = Path(rf"/scratch/slurm-{slurm_id}")
# set_cache_path(Path(rf"/scratch/slurm-{slurm_id}/cache"))

In [ ]:
location = basepath / "simulation_4_311b96fd_10110.sqlite"
location = Path(location)
is_reciprocal = location.stem.split("_")[3][1] != "0"
print(f"is_reciprocal: {is_reciprocal}")

In [ ]:
parameters = {
    "coupling": "properties|::Root Element::R_1|coupling",
    "phase": "properties|::Root Element::R_1|phase",
    "t_1": "results|::Root Element::OSA_R_1_rt|mode 1/signal|values",
    "t_2": f"results|::Root Element::OSA_R_1_{'lb' if is_reciprocal else 'rb'}|mode 1/signal|values",
    "f_1": "results|::Root Element::OSA_R_1_rt|mode 1/signal|Frequency",
    "f_2": f"results|::Root Element::OSA_R_1_{'lb' if is_reciprocal else 'rb'}|mode 1/signal|Frequency",
}

In [ ]:
parameters = get_parameters(location, parameters, False)

In [ ]:
phi_shift = (np.pi / 2) if is_reciprocal else (np.pi / 4)
phi_limits = (0, np.pi) if is_reciprocal else (-np.pi / 2, np.pi / 2)

all_phases = np.unique(parameters["phase"])
using_phi_shift = all_phases[np.argmin(np.abs(all_phases - phi_shift))]
print(f"Phase range: [{np.min(all_phases)}, {np.max(all_phases)}]")
print(f"Using phase shift: {using_phi_shift}")

coupling = 0.1
all_coupling = np.unique(parameters["coupling"])
using_coupling = all_coupling[np.argmin(np.abs(all_coupling - coupling))]
coupling_phi_index = np.where((parameters["coupling"] == using_coupling) & (parameters["phase"] == using_phi_shift))[0][0]
print(f"Coupling range: [{np.min(all_coupling)}, {np.max(all_coupling)}]")
print(f"Using coupling: {using_coupling}")
print(f"Using coupling_phi_index: {coupling_phi_index}")

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(parameters["f_1"][coupling_phi_index], parameters["t_1"][coupling_phi_index], label="Signal 1")
ax.plot(parameters["f_2"][coupling_phi_index], parameters["t_2"][coupling_phi_index], label="Signal 2")
ax.set_xlabel("Frequency (THz)")
ax.set_ylabel("Signal (dBm)")
ax.set_title(
    f"Simulation 2: {location.stem}\n"
    f"{parameters['coupling'][coupling_phi_index]:.3f} coupling, "
    f"{parameters['phase'][coupling_phi_index]:.3f} phase"
)
ax.legend()
plt.tight_layout()
plt.savefig(get_plots_path() / f"{location.stem}_frequency_{int(coupling * 10)}_{int(using_phi_shift * 10 / np.pi)}.png", dpi=600)
plt.show()
plt.close(fig)

In [ ]:
frequency = 3e8 / 1550e-9
signal_1_on_index = np.array(parameters["t_1"][coupling_phi_index])
freq_on_index = np.array(parameters["f_1"][coupling_phi_index])
peaks = np.where(signal_1_on_index < (np.min(signal_1_on_index) * 0.9))[0]
freq_of_peaks = freq_on_index[peaks]
using_freq = freq_of_peaks[np.argmin(np.abs(freq_of_peaks - frequency))]
freq_index = peaks[np.argmin(np.abs(freq_of_peaks - frequency))]

print(f"Using frequency (probe at): {3e8 * 1e9 / using_freq:.3f} nm or {using_freq * 1e-12:.3f} THz")
print(f"Using index: {freq_index}")

In [ ]:
freq_1545 = np.argmin(np.abs(freq_on_index - 3e8 / 1545e-9))
freq_1555 = np.argmin(np.abs(freq_on_index - 3e8 / 1555e-9))
new_freq = freq_on_index[freq_1555:freq_1545]

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(parameters["f_1"][coupling_phi_index][freq_1555:freq_1545], parameters["t_1"][coupling_phi_index][freq_1555:freq_1545], label="Signal 1")
ax.plot(parameters["f_2"][coupling_phi_index][freq_1555:freq_1545], parameters["t_2"][coupling_phi_index][freq_1555:freq_1545], label="Signal 2")
ax.set_xlabel("Frequency (THz)")
ax.set_ylabel("Signal (dBm)")
ax.set_title(f"Simulation 2: {location.stem}\n{parameters['coupling'][coupling_phi_index]:.3f} coupling, {parameters['phase'][coupling_phi_index]:.3f} phase")
ax.legend()
plt.tight_layout()
plt.show()
plt.close(fig)

In [ ]:
if isinstance(parameters["t_1"], np.ndarray):
    parameters["t_1"] = parameters["t_1"].tolist()
if isinstance(parameters["t_2"], np.ndarray):
    parameters["t_2"] = parameters["t_2"].tolist()

In [ ]:
new_parameters = {
    "coupling": all_coupling,
    "phase": all_phases,
    "signal_1": np.zeros((len(all_coupling), len(all_phases), len(new_freq))),
    "signal_2": np.zeros((len(all_coupling), len(all_phases), len(new_freq))),
    "frequencies": new_freq,
}

for i in range(len(parameters["t_1"])):
    if isinstance(parameters["t_1"][i], Sequence):
        sig_1 = parameters["t_1"][i][freq_1555:freq_1545]
    else:
        sig_1 = [parameters["t_1"][i]] * len(new_freq)
    if isinstance(parameters["t_2"][i], Sequence):
        sig_2 = parameters["t_2"][i][freq_1555:freq_1545]
    else:
        sig_2 = [parameters["t_2"][i]] * len(new_freq)
    
    this_coupling = parameters["coupling"][i]
    this_phase = parameters["phase"][i]
    coupling_index = np.where(all_coupling == this_coupling)[0][0]
    phase_index = np.where(all_phases == this_phase)[0][0]
    new_parameters["signal_1"][coupling_index, phase_index] = sig_1
    new_parameters["signal_2"][coupling_index, phase_index] = sig_2

new_parameters["signal_1"] = np.array(new_parameters["signal_1"])
new_parameters["signal_2"] = np.array(new_parameters["signal_2"])
name = "simulation_" + location.name.split("_")[1] + "_" + ("reciprocal" if is_reciprocal else "non_reciprocal") + ".mat"
scipy.io.savemat(get_mat_path() / name, new_parameters)
print("Done")